In [68]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout, Input, GlobalAveragePooling2D
from keras.layers import RandomRotation, RandomFlip, RandomContrast

cc = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/breastcancer/Data/CC',
    labels='inferred',
    label_mode = 'int',
    batch_size=1,
    image_size=(512, 512)
)

mlo = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/breastcancer/Data/MLO',
    labels='inferred',
    label_mode = 'int',
    batch_size=1,
    image_size=(512, 512)
)

Found 100 files belonging to 2 classes.
Found 100 files belonging to 2 classes.


In [77]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, Flatten, Dense, Reshape, Dropout, Add

def residual_block(X,filters):
    # Retrieve Filters
    F1, F2 = filters
    # Saving the input value.we need this later to add to the output. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (3, 3), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(alpha=0.1)(X)

    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = (3, 3), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization()(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = LeakyReLU(alpha=0.1)(X)
    return X

def build_model():
    # Inputs to the model
    base_model = ResNet50(
        weights='imagenet',
        input_shape=(512, 512, 3),  # Input shape of the images (height, width, channels)
        include_top=False  # Exclude the top classification layers
    )

    # Freeze the base model's layers to prevent them from being trained
    base_model.trainable = False
    x = base_model.output
    
    # First conv block
    x = Conv2D(32,(3, 3),kernel_initializer="he_normal",padding="same")(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    # Add a residual block
    x = residual_block(x, [64, 32]) 
    x = residual_block(x, [64, 32]) 
    x = Flatten()(x)
    
    x = Dense(128, kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=x)
    
    return model

In [78]:
# Stage 1
model_1 = build_model()
model_2 = build_model()

In [79]:
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_1.fit(cc,epochs=10)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - accuracy: 0.4796 - loss: 27.1905
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.6174 - loss: 2.5246
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6737 - loss: 1.3267
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6453 - loss: 1.1679
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6817 - loss: 0.6052
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.6988 - loss: 0.6690
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7979 - loss: 0.4838
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7180 - loss: 0.9502
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8703 - loss: 0.3172
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8989 - loss: 0.3232


In [80]:
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_2.fit(mlo,epochs=10)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 27ms/step - accuracy: 0.3887 - loss: 24.3404
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.5762 - loss: 2.5161
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.4786 - loss: 2.2849
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6665 - loss: 1.3708
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6908 - loss: 1.1095
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.6978 - loss: 0.9872
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.6445 - loss: 1.0170
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8190 - loss: 0.4244
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.8117 - loss: 0.3749
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8331 - loss: 0.4783


In [45]:
!mkdir /kaggle/working/outputs

mkdir: cannot create directory '/kaggle/working/outputs': File exists


In [81]:
model_1.save_weights("/kaggle/working/outputs/stage1_cc_weights.weights.h5")
model_2.save_weights("/kaggle/working/outputs/stage1_mlo_weight.weights.h5")

In [83]:
img_path = '/kaggle/input/breastcancer/Data/CC/Normal/10_cc_diff.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(512, 512))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

probabilities_1 = model_1.predict(img_array) 
probabilities_1

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


array([[0.01197649]], dtype=float32)

In [84]:
img_path = '/kaggle/input/breastcancer/Data/MLO/Normal/10_mlo_diff.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(512, 512))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

probabilities_2 = model_2.predict(img_array) 
probabilities_2

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


array([[0.00327079]], dtype=float32)

In [85]:
average_probability = (probabilities_1 + probabilities_2) / 2.0

print("Average Probability:", average_probability)
# If average of probability of class is less than 0.4(threshold), we will classify it as Normal else Suspicious

Average Probability: [[0.00762364]]


In [86]:
# Stage 2
cc = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/classification/stage2/CC',
    labels='inferred',
    label_mode = 'int',
    batch_size=1,
    image_size=(512, 512)
)

mlo = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/classification/stage2/MLO',
    labels='inferred',
    label_mode = 'int',
    batch_size=1,
    image_size=(512, 512)
)

Found 32 files belonging to 2 classes.
Found 32 files belonging to 2 classes.


In [87]:
model_1 = build_model()
model_2 = build_model()

In [88]:
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_1.fit(cc,epochs=10)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - accuracy: 0.5658 - loss: 27.6364
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3881 - loss: 7.5514
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7920 - loss: 4.1345
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8640 - loss: 0.6169
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8715 - loss: 2.1166
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8386 - loss: 1.2617
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7771 - loss: 1.8639
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8904 - loss: 0.3266
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8567 - loss: 0.5641
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9961 - loss: 0.0060    


In [89]:
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_2.fit(mlo,epochs=10)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - accuracy: 0.4996 - loss: 29.9417
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.6080 - loss: 4.8802
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7204 - loss: 3.1897
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8121 - loss: 1.7819
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.6160 - loss: 3.1685
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9316 - loss: 0.1212
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8761 - loss: 0.1979
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8916 - loss: 9.8962 
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9473 - loss: 0.0741
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9381 - loss: 0.4373


In [90]:
model_1.save_weights("/kaggle/working/outputs/stage2_cc_weights.weights.h5")
model_2.save_weights("/kaggle/working/outputs/stage2_mlo_weight.weights.h5")

In [91]:
img_path = '/kaggle/input/classification/stage2/CC/Benign/10_cc_diff.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(512, 512))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

probabilities_1 = model_1.predict(img_array) 
probabilities_1

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


array([[4.3136265e-18]], dtype=float32)

In [92]:
img_path = '/kaggle/input/classification/stage2/MLO/Benign/10_mlo_diff.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(512, 512))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

probabilities_2 = model_2.predict(img_array) 
probabilities_2

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


array([[5.891657e-05]], dtype=float32)

In [93]:
average_probability = (probabilities_1 + probabilities_2) / 2.0

print("Average Probability:", average_probability)
# If average of probability of class is less than 0.5(threshold), we will classify it as Benign else Suspicious

Average Probability: [[2.9458284e-05]]
